## WHISLTER SIMULATED DATASET CLASS
_Represent a dataset of simulated whislter samples_

In [5]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp

class DatasetSimulation(object):
    ''''''
    __t_res, __f_res = None, None
    __f = np.linspace(1e3, 10e3, 1000)
    __fn = 25e3
    
    # Initializer
    def __init__(self, dataset_sim_location, t_res, f_res, ext='.sim'):
        self.__dataset_sim_location = dataset_sim_location
        self.__file_extension = ext
        self.__t_res = t_res
        self.__f_res = f_res
    
    def set_resolution(t,f):
        self.__t_res, self.__f_res = t,f
        
    def whistler_trace(self, An, D0, fn, f):
        """generate the whistler trace
        Params
            An: normalised equatorial electron gyrofrequency
            D0: zero dispersion
            fn: nose frequency
            f: frequency range
        return
            t: time range
            """
        t = (D0/((1+An)*np.sqrt(f)))*(((1+An)-(3*An-1)*(f/fn))/(1-An*f/fn))
        return np.array(t)

    def whistler_sim(self, An, D0, fn, f, t_res, f_res):
        """Generate a 2D representation of the whistler trace
        Params
            An: normalised equatorial electron gyrofrequency
            D0: zero dispersion
            fn: nose frequency
            f: frequency range
        return
            t: time range"""
        t = self.whistler_trace(An=An, D0=D0, fn=fn, f=f)
        t_trans, f_trans = (t-t.min())/t_res, (f-f.min())*1e-3/f_res
        t_trans, f_trans = t_trans.astype(np.int), f_trans.astype(np.int) 
        coor = np.array([(t,f) for t,f in zip(t_trans, f_trans)])
        data = np.zeros((t_trans.max()+1,f_trans.max()+1))
        for x,y in coor:
            data[x,y]=3
        return data.T
    
    def whistler_file(self, An, D0):
        An = np.round(An,4)
        
        data = self.whistler_sim(An=An, D0=D0, fn=self.__fn, f=self.__f, t_res=self.__t_res, f_res=self.__f_res)
        file_name = os.path.join(self.__dataset_sim_location, str(An)+'_'+str(D0)+'_'+str(self.__fn)+self.__file_extension)
        file = h5py.File(file_name, 'w')
        file.create_dataset(file_name, data.shape, np.float32, compression='gzip', data=data)
        file.close()
    
    def load_whistler_file(self, An, D0):
        file_name = os.path.join(self.__dataset_sim_location, str(An)+'_'+str(D0)+'_'+str(self.__fn)+self.__file_extension)
        try:
            file = h5py.File(file_name, 'r+')
        except Exception as e:
            print('File: %s. Exception: %s'%(file,e))
        file_data = np.empty(file[file_name].shape,dtype=np.float64)
        file[file_name].read_direct(file_data)
        data = np.asarray(file_data)
        file.close()
        return data
    
    def whistler_file_(self,param):
            self.whistler_file(An=param[0], D0=param[1])
    
    def generate_whislters(self, An_dec=4, D0_dec=2):
        An_min, An_max = 0.35, 0.45
        An = np.linspace(An_min, An_max, int((An_max-An_min)/10**(-An_dec))+1)
        D0_min, D0_max = 20, 80
        D0 = np.linspace(D0_min, D0_max, int((D0_max-D0_min)/10**(-D0_dec))+1)
        params = [(a,d) for a in An for d in D0] # define parameters
        # use multiprocessing
        pool = mp.Pool(mp.cpu_count())
        for _ in tqdm(pool.imap_unordered(self.whistler_file_, params), total=len(params)):
            pass
        pool.close()

In [4]:
# dataset_sim_loc = os.path.join(os.getcwd().split(os.environ.get('USER'))[0],os.environ.get('USER'), 'wdml', 'data','datasets', 'simulations')
# dataset_sim = DatasetSimulation(dataset_sim_loc, 0.006395061728395062, 0.15503875968992248)
# # dataset_sim.whistler_file(An=0.35, D0=20)
# # plt.pcolormesh(dataset_sim.load_whistler_file(An=0.35, D0=20))
# dataset_sim.generate_whislters()


  0%|          | 0/2 [00:00<?, ?it/s]

0.35 20.0


0.35 21.0



100%|██████████| 2/2 [00:00<00:00,  3.59it/s]

In [52]:
a = [1,2]
b = ['a', 'b', 'c']
[(i,j) for i in a for j in b]

[(1, 'a'), (1, 'b'), (1, 'c'), (2, 'a'), (2, 'b'), (2, 'c')]